<a href="https://colab.research.google.com/github/Yagomendonca/Dashboard_Phyton/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import das bibliotecas

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import plotly.express as px


import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import scale, minmax_scale, power_transform

### Import do Dataframe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CURSO ANALISE DE DADOS DNC/MÊS 08/Desafio 5/data.csv', encoding='unicode_escape')

In [ ]:
df

### Entendendo os dados

Mudança no nome das colunas

In [ ]:
df = df.rename(columns={'InvoiceNo':'Codigo_da_fatura', 'StockCode':'Codigo_de_estoque', 'InvoiceDate':'Data_do_Faturamento'})

Verificação dos dados

In [ ]:
df.describe()

In [ ]:
df.max()

In [ ]:
df.info()

### Limpeza dos dados

Verificação dos dados nulos

In [ ]:
df.isnull().sum()

Entendendo que o Customer ID é igual ao código de identificação do cliente, não há motivos de se ter códigos vazios, pois se o código é vazio não exite cliente, logo, a venda não foi realizada, ou foi realizada para um cliente fantasma

In [ ]:
df[df['CustomerID'].isna()]

In [ ]:
df[df['Description'].isna()]

Aqui entendeu-se que algumas descrições dos produtos são nulas quando o Customer ID é nulo. Logo, foi necessário excluir apenas os dados em que o Customer ID é nulo para que as descrições tambem sejam excluídas e assim o dataset ficará sem valores nulos

In [ ]:
df.dropna(subset=['CustomerID'], inplace=True)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df = df.query('UnitPrice > 0')

In [ ]:
df = df.query('Quantity > 0')

In [ ]:
df

In [ ]:
df.describe()

Exclusão de valores <= 0

In [ ]:
df.duplicated().sum()

In [ ]:
df[df.duplicated()]

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.duplicated().sum()

Exclusão de valores duplicados

In [ ]:
df.info()

In [ ]:
df.max()

### Mudando o tipo dos dados

In [ ]:
df['Data_do_Faturamento'] =  df['Data_do_Faturamento'].astype('datetime64')

In [ ]:
df

In [ ]:
df['CustomerID'] = df['CustomerID'].astype(int)

In [ ]:
df.info()

### Tratando os outliers

In [ ]:
df.describe()

In [ ]:
df.plot.box()

In [ ]:
df[['UnitPrice', 'Quantity']].apply(scale).boxplot()

In [ ]:
df.query('UnitPrice>=5000')

In [ ]:
df.query('Quantity>10000')

In [ ]:
df.describe(percentiles=[.01, .25, .5, .75, .9, .99])

In [ ]:
def is_outlier(array, extreme=False):
  """Custom function to identify outliers in an array"""
  q1,q3 = np.quantile(array, [.25,.99])
  iqr = q3-q1

  factor = 400. if extreme else 350
  upper_outlier = q3 + factor*iqr
  lower_outlier = q1 - factor*iqr

  return (array < lower_outlier) | (array > upper_outlier)

In [ ]:
df[['UnitPrice', 'Quantity']].apply(is_outlier).sum()

In [ ]:
df= df[~df[['UnitPrice', 'Quantity']].apply(is_outlier).any(axis=1)]

In [ ]:
df.query('UnitPrice>=5000')
df.query('Quantity>=10000')

### Análises matemáticas

Verificando o valor total em vendas

In [ ]:
df['Preço_Total'] = df['Quantity'] * df['UnitPrice']

In [ ]:
df.head()

In [ ]:
df.info()

Verificando a ultima data

In [ ]:
df.sort_values('Data_do_Faturamento', ascending=False)

Calculando o top 10 países com maior valor em vendas

In [ ]:
Top10Países = df[['Country', 'Preço_Total']].groupby('Country').sum().sort_values('Preço_Total',ascending=False).head(10)

In [ ]:
Top10Países

In [ ]:
Top10Países.to_excel('Top_10_Paises.xlsx')

Top 10 produtos com maior valor em vendas

In [ ]:
Top10Produtos = df[['Description', 'Quantity']].groupby('Description').sum('Quantity').sort_values('Quantity', ascending=False).head(10)

In [ ]:
Top10Produtos

In [ ]:
Top10Produtos.to_excel('Top_10_Produtos.xlsx')

Venda total po mês

In [ ]:
UltimaData = df['Data_do_Faturamento'].max()

In [ ]:
UltimaData

In [ ]:
df_date_index = df.set_index('Data_do_Faturamento').sort_index()
df_date_index

In [ ]:
Venda_Total_Mes = df_date_index[['Preço_Total']].resample('MS').sum()

In [ ]:
Venda_Total_Mes

In [ ]:
Venda_Total_Mes.to_excel('Venda_mes.xlsx')

Venda total por país e mês

In [ ]:
Venda_Total_Mes2 = df.groupby([df['Data_do_Faturamento'].dt.strftime('%Y-%m'), 'Country'])['Preço_Total'].sum().reset_index()

In [ ]:
Venda_Total_Mes2

In [ ]:
Venda_Total_Pais_Mes = Venda_Total_Mes2.groupby('Country')['Preço_Total'].sum().nlargest(10).index
df_Top10 = Venda_Total_Mes2[Venda_Total_Mes2['Country'].isin(Venda_Total_Pais_Mes)].sort_values('Preço_Total', ascending=False)

In [ ]:
df_Top10

In [ ]:
df_Top10.to_excel('Top_10.xlsx')

### Dashboard

In [ ]:
!pip install dash

In [ ]:
from dash import Dash, html, dcc

In [ ]:
fig_Top10Países = px.bar(Top10Países, color_discrete_sequence=px.colors.qualitative.Alphabet, title = 'Top 10 países')
fig_Top10Países.update_layout(title=dict(text='Top 10 Países', font=dict(size=35), xanchor = 'auto', yanchor ="top"))
fig_Top10Países.show();
fig_Top10Produtos = px.bar(Top10Produtos, y ='Quantity', color_discrete_sequence=px.colors.qualitative.Alphabet, title = 'Top 10 produtos')
fig_Top10Produtos.update_layout(title=dict(text='Top 10 Produtos', font=dict(size=35), xanchor = 'auto', yanchor ="top"))
fig_Top10Produtos.show();
fig_Venda_Total_Mes = px.area(Venda_Total_Mes, color_discrete_sequence=px.colors.qualitative.Alphabet, title = 'Venda total mês')
fig_Venda_Total_Mes.update_layout(title=dict(text='Venda Total Mes', font=dict(size=35), xanchor = 'auto', yanchor ="top"))
fig_Venda_Total_Mes.show()
fig_Venda_País_Mês = px.bar(df_Top10, x='Data_do_Faturamento', y='Preço_Total', color_discrete_sequence=px.colors.qualitative.Alphabet, color='Country', title = 'Top 10 País por mês ')
fig_Venda_País_Mês.update_layout(title=dict(text='Venda País Mês', font=dict(size=35), xanchor = 'auto', yanchor ="top"))

fig_Venda_País_Mês.show()

### Cálculo do RFM

In [ ]:
df.groupby(['CustomerID', 'Description', 'Data_do_Faturamento'])[['Preço_Total']].mean()

Cálculo da Recência

In [ ]:
df.groupby(['CustomerID'])[['Preço_Total']].mean()

In [ ]:
Recencia = df.groupby(['CustomerID'])[['Data_do_Faturamento']].max()

In [ ]:
Recencia.info()

In [ ]:
UltimaData

In [ ]:
Recencia.query('12347')

In [ ]:
Recencia['Dias'] = abs(Recencia - UltimaData)

In [ ]:
Recencia.info()

In [ ]:
Recencia

Cálculo da Frequência

In [ ]:
Frequencia = df.groupby(['CustomerID'])[['Quantity']].count()

In [ ]:
Frequencia.sort_values('Quantity',ascending=False)

Cálculo do Ticket Médio

In [ ]:
Ticket_medio = df.groupby(['CustomerID'])[['Preço_Total']].mean()

In [ ]:
Ticket_medio.sort_values('Preço_Total', ascending=False)